In [1]:

# Now import your custom modules
import bloomberg_data as bd
import transformations as tr
import visuals as vis



import vectorbt as vbt
import numpy as np
import pandas as pd
import datetime
import plotly.express as px
from xbbg import blp
import os
import quantstats as qs
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import logging





In [11]:

# Main data retrieval and merging process
tickers = ['.MIDERCAD U Index', 'GCAN3M   Index']
fields = [['PX_LAST'], ['PX_LAST']]
start_date = '2000-01-01'
end_date = '2025-12-31'
column_names = [['cad_ig_er_index'], ['risk_free']]
frequency = 'd'  # Single frequency for all tickers

dataframes = []

for ticker, field, col_name in zip(tickers, fields, column_names):
    df = bd.get_single_ticker_data(ticker, field, start_date, end_date, freq=frequency, column_names=col_name)
    dataframes.append(df)
    logging.info(f"Data for {ticker}:")
    logging.info(df.head())  # Print the first few rows of each dataframe

# Merge all dataframes
merged_data = bd.merge_dataframes(dataframes, method='outer')

# Print the final merged data and its information
logging.info("Merged data head:")
logging.info(merged_data.head())
logging.info('----------------------------------------------------------------')
logging.info('----------------------------------------------------------------')
logging.info(merged_data.tail())
logging.info(merged_data.info())

# Rename the index to "Date" and reset it
merged_data.index.name = 'Date'
csv_data = merged_data.reset_index()

# Save the dataframe to a CSV file
csv_data.to_csv('Outputs/csv_data.csv', index=False)

# Rename for further use
data = merged_data

2024-09-23 13:19:23,386 - INFO - Retrieving data for ticker: .MIDERCAD U Index with frequency: DAILY
2024-09-23 13:19:24,269 - INFO - Retrieved data shape for .MIDERCAD U Index: (2137, 1)
2024-09-23 13:19:24,270 - INFO - Cleaned data shape for .MIDERCAD U Index: (2137, 1)
2024-09-23 13:19:24,271 - INFO - Successfully retrieved data for ticker: .MIDERCAD U Index
2024-09-23 13:19:24,271 - INFO - Data for .MIDERCAD U Index:
2024-09-23 13:19:24,271 - INFO -             cad_ig_er_index
2002-11-29           1.0143
2002-12-31           1.0146
2003-01-31           1.0155
2003-02-28           1.0159
2003-03-31           1.0142
2024-09-23 13:19:24,272 - INFO - Retrieving data for ticker: GCAN3M   Index with frequency: DAILY
2024-09-23 13:19:24,661 - INFO - Retrieved data shape for GCAN3M   Index: (5963, 1)
2024-09-23 13:19:24,662 - INFO - Cleaned data shape for GCAN3M   Index: (5963, 1)
2024-09-23 13:19:24,662 - INFO - Successfully retrieved data for ticker: GCAN3M   Index
2024-09-23 13:19:24,66

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5975 entries, 2000-01-06 to 2024-09-23
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cad_ig_er_index  5975 non-null   float64
 1   risk_free        5975 non-null   float64
dtypes: float64(2)
memory usage: 140.0 KB


In [12]:
data

,cad_ig_er_index,risk_free
Date,,
2000-01-06,1.0143,4.973
2000-01-07,1.0143,4.912
2000-01-10,1.0143,4.991
2000-01-11,1.0143,4.966
2000-01-12,1.0143,4.967
...,...,...
2024-09-17,1.3925,4.051
2024-09-18,1.3935,4.037
2024-09-19,1.3952,4.042


In [13]:
print(merged_data[['risk_free']].index)

DatetimeIndex(['2000-01-06', '2000-01-07', '2000-01-10', '2000-01-11',
               '2000-01-12', '2000-01-13', '2000-01-14', '2000-01-17',
               '2000-01-18', '2000-01-19',
               ...
               '2024-09-10', '2024-09-11', '2024-09-12', '2024-09-13',
               '2024-09-16', '2024-09-17', '2024-09-18', '2024-09-19',
               '2024-09-20', '2024-09-23'],
              dtype='datetime64[ns]', name='Date', length=5975, freq=None)


In [14]:
# Calculate the compounded risk-free index
risk_free_index = tr.risk_free_index(merged_data[['risk_free']], col_name='3m_tbill_index')
risk_free_index

2024-09-23 13:19:32,173 - WARNING - Cannot infer frequency. Attempting to resample to daily frequency.
2024-09-23 13:19:32,178 - INFO - Inferred frequency: D


,3m_tbill_index_index
Date,
2000-01-06,100.000000
2000-01-07,100.013458
2000-01-08,100.026917
2000-01-09,100.040378
2000-01-10,100.054058
...,...
2024-09-19,166.163490
2024-09-20,166.181900
2024-09-21,166.200312


In [15]:
# Example use of the get_er_index function:
levered_er_index= tr.get_er_index(merged_data[['cad_ig_er_index']], cost_of_borrow=0.40, leverage=3, col_name="levered_er")
levered_er_index


2024-09-23 13:19:35,414 - WARNING - Cannot infer frequency. Attempting to resample to daily frequency.
2024-09-23 13:19:35,415 - INFO - Inferred frequency: D


,levered_er_index
Date,
2000-01-06,100.000000
2000-01-07,99.999967
2000-01-08,99.999934
2000-01-09,99.999901
2000-01-10,99.999868
...,...
2024-09-19,239.752175
2024-09-20,240.112961
2024-09-21,240.112882


In [16]:
credit_fund_estimated_index= tr.get_credit_fund_estimated_index(risk_free_index,levered_er_index,"credit_fund_estimated_index")
credit_fund_estimated_index

,credit_fund_estimated_index
Date,
2000-01-06,100.000000
2000-01-07,100.013425
2000-01-08,100.026851
2000-01-09,100.040279
2000-01-10,100.053926
...,...
2024-09-19,398.378604
2024-09-20,399.022235
2024-09-21,399.066313


In [17]:
credit_returns=bd.merge_dataframes([data,risk_free_index,levered_er_index,credit_fund_estimated_index], method='outer')
# Drop the 'risk_free' column from the DataFrame
credit_returns = credit_returns.drop(columns=['risk_free'])

# Save the DataFrame to a CSV file in the Outputs folder
credit_returns.to_csv('Outputs/credit_returns.csv', index=True)

credit_returns



2024-09-23 13:19:58,467 - INFO - Merged 4 dataframes using outer method.


,cad_ig_er_index,3m_tbill_index_index,levered_er_index,credit_fund_estimated_index
Date,,,,
2000-01-06,1.0143,100.000000,100.000000,100.000000
2000-01-07,1.0143,100.013458,99.999967,100.013425
2000-01-08,1.0143,100.026917,99.999934,100.026851
2000-01-09,1.0143,100.040378,99.999901,100.040279
2000-01-10,1.0143,100.054058,99.999868,100.053926
...,...,...,...,...
2024-09-19,1.3952,166.163490,239.752175,398.378604
2024-09-20,1.3959,166.181900,240.112961,399.022235
2024-09-21,1.3959,166.200312,240.112882,399.066313


In [18]:
tr.get_cagr(credit_returns)

2024-09-23 13:20:04,299 - INFO - Inferred frequency: D


,cad_ig_er_index,3m_tbill_index_index,levered_er_index,credit_fund_estimated_index
CAGR,0.013005,0.020778,0.036078,0.057605


In [10]:
# Import necessary libraries
import pandas as pd
import numpy as np
import datetime
import logging
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Import custom modules with an alias
import bloomberg_data as bd
import transformations as tr
import visuals as vis

# Set up logging
logging.basicConfig(level=logging.INFO)

# Main data retrieval and merging process
tickers = ['.MIDERCAD U Index', 'GCAN3M   Index']
fields = [['PX_LAST'], ['PX_LAST']]
start_date = '2000-01-01'
end_date = '2025-12-31'
column_names = [['cad_ig_er_index'], ['risk_free']]
frequency = 'M'  # Single frequency for all tickers

# Retrieve and merge data
dataframes = []

for ticker, field, col_name in zip(tickers, fields, column_names):
    df = bd.get_single_ticker_data(ticker, field, start_date, end_date, freq=frequency, column_names=col_name)
    dataframes.append(df)
    logging.info(f"Data for {ticker}:")
    logging.info(df.head())  # Print the first few rows of each dataframe

# Merge all dataframes
merged_data = bd.merge_dataframes(dataframes, method='outer')

# Print the final merged data and its information
logging.info("Merged data head:")
logging.info(merged_data.head())
logging.info('----------------------------------------------------------------')
logging.info('----------------------------------------------------------------')
logging.info(merged_data.tail())
logging.info(merged_data.info())

# Rename the index to "Date" and reset it
merged_data.index.name = 'Date'
csv_data = merged_data.reset_index()

# Save the dataframe to a CSV file
csv_data.to_csv('Outputs/csv_data.csv', index=False)

# Rename for further use
data = merged_data

# Calculate the compounded risk-free index
risk_free_index = tr.risk_free_index(merged_data[['risk_free']], col_name='3m_tbill_index')

# Example use of the get_er_index function
levered_er_index = tr.get_er_index(merged_data[['cad_ig_er_index']], cost_of_borrow=0.40, leverage=3, col_name="levered_er")

# Credit fund estimated index
credit_fund_estimated_index = tr.get_credit_fund_estimated_index(risk_free_index, levered_er_index, "credit_fund_estimated_index")

# Credit returns
credit_returns = bd.merge_dataframes([data, risk_free_index, levered_er_index, credit_fund_estimated_index], method='outer')

# Drop the 'risk_free' column from the DataFrame
credit_returns = credit_returns.drop(columns=['risk_free'])

# Save the DataFrame to a CSV file in the Outputs folder
credit_returns.to_csv('Outputs/credit_returns.csv', index=True)

# Print the credit returns DataFrame
print(credit_returns)

# Calculate the CAGR of credit returns
cagrs = tr.get_cagr(credit_returns)

# Print the CAGR of credit returns
print(cagrs)


2024-09-23 13:17:22,824 - INFO - Retrieving data for ticker: .MIDERCAD U Index with frequency: MONTHLY
2024-09-23 13:17:23,717 - INFO - Retrieved data shape for .MIDERCAD U Index: (263, 1)
2024-09-23 13:17:23,718 - INFO - Cleaned data shape for .MIDERCAD U Index: (263, 1)
2024-09-23 13:17:23,720 - INFO - Successfully retrieved data for ticker: .MIDERCAD U Index
2024-09-23 13:17:23,720 - INFO - Data for .MIDERCAD U Index:
2024-09-23 13:17:23,720 - INFO -             cad_ig_er_index
2002-11-30           1.0143
2002-12-31           1.0146
2003-01-31           1.0155
2003-02-28           1.0159
2003-03-31           1.0142
2024-09-23 13:17:23,721 - INFO - Retrieving data for ticker: GCAN3M   Index with frequency: MONTHLY
2024-09-23 13:17:23,984 - INFO - Retrieved data shape for GCAN3M   Index: (297, 1)
2024-09-23 13:17:23,984 - INFO - Cleaned data shape for GCAN3M   Index: (297, 1)
2024-09-23 13:17:23,986 - INFO - Successfully retrieved data for ticker: GCAN3M   Index
2024-09-23 13:17:23,98

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 297 entries, 2000-01-31 to 2024-09-30
Freq: ME
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cad_ig_er_index  297 non-null    float64
 1   risk_free        297 non-null    float64
dtypes: float64(2)
memory usage: 15.1 KB
            cad_ig_er_index  3m_tbill_index_index  levered_er_index  \
Date                                                                  
2000-01-31           1.0143            100.000000        100.000000   
2000-02-29           1.0143            100.402667         99.999000   
2000-03-31           1.0143            100.844104         99.998000   
2000-04-30           1.0143            101.298743         99.997000   
2000-05-31           1.0143            101.777970         99.996000   
...                     ...                   ...               ...   
2024-05-31           1.3949            163.589116        234.982370   
2024

In [44]:
# getting ths er index from mtd bloomberg

# Main data retrieval and merging process
tickers = ['I05510CA Index','LUACTRUU Index']
fields = [['INDEX_EXCESS_RETURN_YTD'], ['INDEX_EXCESS_RETURN_YTD']]
start_date = '2000-01-01'
end_date = '2025-12-31'
column_names = [['cad_ig_er_calc'], ['us_ig_er_calc']]
frequency = 'd'  # Single frequency for all tickers

dataframes = []

for ticker, field, col_name in zip(tickers, fields, column_names):
    df = bd.get_single_ticker_data(ticker, field, start_date, end_date, freq=frequency, column_names=col_name)
    dataframes.append(df)
    logging.info(f"Data for {ticker}:")
    logging.info(df.head())  # Print the first few rows of each dataframe

# Merge all dataframes
merged_data = bd.merge_dataframes(dataframes, method='outer')

# Print the final merged data and its information
logging.info("Merged data head:")
logging.info(merged_data.head())
logging.info('----------------------------------------------------------------')
logging.info('----------------------------------------------------------------')
logging.info(merged_data.tail())
logging.info(merged_data.info())


# Rename for further use
er_bloomberd_daily_ytd = merged_data
er_bloomberd_daily_ytd


2024-09-23 15:18:40,348 - INFO - Retrieving data for ticker: I05510CA Index with frequency: DAILY
2024-09-23 15:18:40,764 - INFO - Retrieved data shape for I05510CA Index: (5513, 1)
2024-09-23 15:18:40,765 - INFO - Cleaned data shape for I05510CA Index: (5513, 1)
2024-09-23 15:18:40,766 - INFO - Successfully retrieved data for ticker: I05510CA Index
2024-09-23 15:18:40,767 - INFO - Data for I05510CA Index:
2024-09-23 15:18:40,767 - INFO -             cad_ig_er_calc
2002-10-01        0.094303
2002-10-02       -0.093446
2002-10-03       -0.023706
2002-10-04        0.024013
2002-10-07       -0.061681
2024-09-23 15:18:40,768 - INFO - Retrieving data for ticker: LUACTRUU Index with frequency: DAILY
2024-09-23 15:18:41,129 - INFO - Retrieved data shape for LUACTRUU Index: (6200, 1)
2024-09-23 15:18:41,129 - INFO - Cleaned data shape for LUACTRUU Index: (6200, 1)
2024-09-23 15:18:41,130 - INFO - Successfully retrieved data for ticker: LUACTRUU Index
2024-09-23 15:18:41,131 - INFO - Data for L

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6200 entries, 2000-01-03 to 2024-09-20
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   cad_ig_er_calc  6200 non-null   float64
 1   us_ig_er_calc   6200 non-null   float64
dtypes: float64(2)
memory usage: 145.3 KB


,cad_ig_er_calc,us_ig_er_calc
2000-01-03,0.094303,-0.033030
2000-01-04,0.094303,-0.065850
2000-01-05,0.094303,-0.083246
2000-01-06,0.094303,-0.106658
2000-01-07,0.094303,-0.109214
...,...,...
2024-09-16,1.389579,1.039402
2024-09-17,1.465869,1.200475
2024-09-18,1.488111,1.327045
2024-09-19,1.600825,1.511344


In [45]:
er_bloomberd_daily_ytd.reset_index()
er_bloomberd_daily_ytd.index.name = 'Date'
er_bloomberd_daily_ytd


,cad_ig_er_calc,us_ig_er_calc
Date,,
2000-01-03,0.094303,-0.033030
2000-01-04,0.094303,-0.065850
2000-01-05,0.094303,-0.083246
2000-01-06,0.094303,-0.106658
2000-01-07,0.094303,-0.109214
...,...,...
2024-09-16,1.389579,1.039402
2024-09-17,1.465869,1.200475
2024-09-18,1.488111,1.327045


In [46]:
import pandas as pd
import logging

def convert_er_ytd_to_index(df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts year-to-date excess return data in all columns to custom indices starting at 100.
    Assumes that all columns contain year-to-date excess return data in percentage format.

    :param df: DataFrame containing year-to-date excess return data in percentage format.
    :return: DataFrame with custom indices columns calculated from daily returns.
    """

    # Setup logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    
    # 1. Ensure the DataFrame has a datetime index
    def ensure_datetime_index(dataframe: pd.DataFrame) -> pd.DataFrame:
        """Ensures that the DataFrame has a datetime index. Converts 'Date' column to index if necessary."""
        if 'Date' in dataframe.columns:
            dataframe['Date'] = pd.to_datetime(dataframe['Date'], errors='coerce')
            dataframe.set_index('Date', inplace=True)
        if not isinstance(dataframe.index, pd.DatetimeIndex):
            raise ValueError("The DataFrame must have a datetime index or a 'Date' column.")
        return dataframe

    # 2. Convert returns from percentage to decimal
    def convert_returns_format(dataframe: pd.DataFrame) -> pd.DataFrame:
        """Converts all columns from percentage to decimal format (divide by 100)."""
        dataframe = dataframe / 100
        return dataframe

    # 3 & 4. Calculate daily returns from YTD returns
    def calculate_daily_returns(ytd_returns: pd.Series) -> pd.Series:
        """Calculates daily returns from YTD returns for each year."""
        daily_returns = pd.Series(index=ytd_returns.index, dtype=float)
        
        for year in ytd_returns.index.year.unique():
            year_data = ytd_returns[ytd_returns.index.year == year]
            
            # Handle first day of the year
            daily_returns.loc[year_data.index[0]] = year_data.iloc[0]
            
            # Calculate daily returns for the rest of the year
            daily_returns.loc[year_data.index[1:]] = (1 + year_data.iloc[1:].values) / (1 + year_data.iloc[:-1].values) - 1
        
        return daily_returns

    # 5. Create custom indices starting at 100
    def create_custom_index(daily_returns: pd.Series, start_value: float = 100) -> pd.Series:
        """Creates a custom index starting at a specified value from daily returns."""
        custom_index = start_value * (1 + daily_returns).cumprod()
        return custom_index

    # Process the DataFrame
    df = ensure_datetime_index(df)
    df = convert_returns_format(df)

    index_columns = {}

    # Calculate daily returns and create custom indices for each column
    for column in df.columns:
        daily_returns = calculate_daily_returns(df[column])
        index_column = create_custom_index(daily_returns)
        index_columns[f"{column}_index"] = index_column

    # Create a new DataFrame with the custom indices
    index_df = pd.DataFrame(index_columns, index=df.index)

    return index_df


In [47]:
credit_unlevered_indicies_from_bloomberg_ytd= convert_er_ytd_to_index(er_bloomberd_daily_ytd)
credit_unlevered_indicies_from_bloomberg_ytd.info()



<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6200 entries, 2000-01-03 to 2024-09-20
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   cad_ig_er_calc_index  6200 non-null   float64
 1   us_ig_er_calc_index   6200 non-null   float64
dtypes: float64(2)
memory usage: 274.4 KB


In [48]:
tr.get_cagr(credit_unlevered_indicies_from_bloomberg_ytd)

2024-09-23 15:20:22,125 - WARNING - Cannot infer frequency. Attempting to resample to daily frequency.
2024-09-23 15:20:22,127 - INFO - Inferred frequency: D


,cad_ig_er_calc_index,us_ig_er_calc_index
CAGR,0.011409,0.008163
